In [1]:
import csv

def transform_text(original_text):
    # Find the position of the '<assistant>:' tag
    assistant_pos = original_text.find('<assistant>:')
    if assistant_pos == -1:
        return original_text  # Return original if the format is not as expected

    # Extract question and answer
    question = original_text[len('<human>: '):assistant_pos].strip()
    answer = original_text[assistant_pos + len('<assistant>: '):].strip()

    # Format into the new structure with <HUMAN> and <ASSISTANT>, including a newline
    transformed = f"<HUMAN>: {question}\n<ASSISTANT>: {answer}"
    return transformed

def process_csv(input_file, output_file):
    with open(input_file, 'r', newline='', encoding='utf-8') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile:

        reader = csv.DictReader(infile)
        fieldnames = ['text']  # Assuming 'text' is the only field
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)

        writer.writeheader()

        for row in reader:
            # Skip the header row
            if row['text'].startswith('<human>:'):
                transformed_text = transform_text(row['text'])
                writer.writerow({'text': transformed_text})

# Replace 'input.csv' and 'output.csv' with your actual file names
process_csv('instruct-data.csv', 'falcon-instruct-data.csv')


#Confirm if the format is correct

In [2]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training
from trl import SFTTrainer
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = load_dataset("ZahrizhalAli/mental_health_conversational_dataset")
data

# Take a Glance on how the data looks like 
print(data["train"][0]['text'])

<HUMAN>: What is a panic attack?
<ASSISTANT>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.


In [4]:
from datasets import load_dataset

# Load the dataset from the formatted CSV file
data = load_dataset('csv', data_files='falcon-instruct-data.csv')
# dataset = load_dataset('csv', data_files='owasp-formatted.csv')

# Accessing the dataset
# print(repr(data[0]['text']))  # Print the first row of the dataset
# print(repr(data["train"][0]['text']))
print(data["train"][0]['text'])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

<HUMAN>: What are the top 10 API security risks listed in the OWASP 2023 report, and can you provide a brief description of each?
<ASSISTANT>: The OWASP Top 10 API Security Risks for 2023 are as follows:

API1:2023 Broken Object Level Authorization - This risk involves insufficient controls at the object level which allows unauthorized users to access or modify data.

API2:2023 Broken Authentication - This pertains to vulnerabilities in the authentication mechanisms, allowing attackers to assume other users' identities.

API3:2023 Broken Object Property Level Authorization - Similar to object level authorization but more specific to the properties of objects, where users can access or modify properties without proper authorization.

API4:2023 Unrestricted Resource Consumption - This risk involves the API allowing uncontrolled consumption of resources, which can lead to denial of service (DoS) attacks.

API5:2023 Broken Function Level Authorization - This occurs when function level perm